# Alignement automatique facile

Ce notebook permet d'aligner les audios issuent des sous dossiers d'un dossier principal

# Montage de google Drive

In [1]:
#Google Drive
from google.colab import drive

# 1. Monter Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_ELFA_NLP


/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_ELFA_NLP


In [25]:
!git push -u origin main

error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/foupouagnigni/ELFA_NLP.git'


In [31]:
!git push --set-upstream origin main


error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/foupouagnigni/ELFA_NLP.git'


Specification de l'emplacement du dossier racine : /alignement

In [ ]:
root_path=input("Entrer le chemin du dossier racine : ")

Entrer le chemin du dossier racine : /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/not_verified/extracteur_données_audios/alignement


# Alignement

## Functions

In [ ]:
import os
import shutil
from collections import defaultdict # Import defaultdict for easier counting


def log_message(log_dir, filename, message):
    """
    Writes a message to a log file within the specified log directory.
    Creates the log directory if it doesn't exist.
    """
    os.makedirs(log_dir, exist_ok=True) # Ensure log directory exists
    log_file_path = os.path.join(log_dir, filename)
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(message + "\n")
    print(f"Log message written to: {log_file_path}") # Optional: uncomment for verbose logging


def lister_dossiers(parent_dossier):
    """Lists full paths of subdirectories directly within a parent folder."""
    full_paths = [os.path.join(parent_dossier, d) for d in os.listdir(parent_dossier)]
    return [d for d in full_paths if os.path.isdir(d)]

def lister_fichiers(dossier):
    """Lists full paths of files directly within a folder."""
    full_paths = [os.path.join(dossier, f) for f in os.listdir(dossier)]
    return [f for f in full_paths if os.path.isfile(f)]


# --- Main Alignment Function ---
def alignement(data_dir, aligned_data_dir, log_dir):

    print("Alignment processing ....")
    log_message(log_dir, "alignement.txt", "Alignment processing started.")

    #Use defaultdict to automatically initialize counts to 0
    # This will store {base_filename: count_of_its_occurrences_across_subdirs}
    file_occurrence_counts = defaultdict(int)

    #Store full paths of source subdirectories
    source_sub_dirs_full_paths = lister_dossiers(data_dir)
    num_expected_occurrences = len(source_sub_dirs_full_paths)

    if num_expected_occurrences == 0:
        print(f"Error: No subdirectories found in '{data_dir}'. Alignment aborted.")
        log_message(log_dir, "alignement_errors.txt", f"Error: No subdirectories found in '{data_dir}'. Alignment aborted.")
        return

    print(f"Found {num_expected_occurrences} source subdirectories in '{data_dir}'.")
    log_message(log_dir, "alignement.txt", f"Found {num_expected_occurrences} source subdirectories.")

    # --- First Pass: Scan all subdirectories to count file occurrences ---
    for sub_dir_full_path in source_sub_dirs_full_paths:
        print(f"Scanning subdirectory: '{sub_dir_full_path}'")
        files_in_current_sub_dir = lister_fichiers(sub_dir_full_path)

        for file_full_path in files_in_current_sub_dir:
            file_base_name = os.path.basename(file_full_path)
            file_occurrence_counts[file_base_name] += 1

    # --- Second Pass: Identify truly aligned files (present in ALL subdirectories) ---
    aligned_common_files_base_names = {
        base_name: count
        for base_name, count in file_occurrence_counts.items()
        if count == num_expected_occurrences
    }

    if not aligned_common_files_base_names:
        print("No files found common to all subdirectories. Alignment finished.")
        log_message(log_dir, "alignement.txt", "No files found common to all subdirectories. Alignment finished.")
        return

    print(f"\nFound {len(aligned_common_files_base_names)} files common to all {num_expected_occurrences} subdirectories. Starting copy process...")
    log_message(log_dir, "alignement.txt", f"Identified common files: {list(aligned_common_files_base_names.keys())}")


    # --- Third Pass: Copy the identified common files to the aligned_data_dir ---
    for sub_dir_full_path in source_sub_dirs_full_paths:
        # Get just the subdirectory name to recreate the structure in the output
        sub_dir_name_only = os.path.basename(sub_dir_full_path)
        output_sub_dir = os.path.join(aligned_data_dir, sub_dir_name_only)

        print(f"Ensuring output directory exists: '{output_sub_dir}'")
        os.makedirs(output_sub_dir, exist_ok=True) # `exist_ok=True` prevents errors if directory already exists

        for common_file_base_name in aligned_common_files_base_names:
            # Construct the full source path for the common file in the current source subdirectory
            source_file_path = os.path.join(sub_dir_full_path, common_file_base_name)
            destination_file_path = os.path.join(output_sub_dir, common_file_base_name)

            if os.path.exists(source_file_path):
                try:
                    shutil.copy(source_file_path, destination_file_path)
                    print(f"Copied: '{source_file_path}' to '{destination_file_path}'")
                    log_message(log_dir, "alignement.txt", f"Copied: {source_file_path} -> {destination_file_path}")
                except Exception as e:
                    print(f"Error copying '{source_file_path}' to '{destination_file_path}': {e}")
                    log_message(log_dir, "alignement_errors.txt", f"Error copying {source_file_path} to {destination_file_path}: {e}")
            else:
                # This case should ideally not happen if 'aligned_common_files_base_names' is correct,
                # but it's a good defensive check.
                print(f"Warning: Common file '{common_file_base_name}' not found in '{sub_dir_full_path}'. Skipping copy for this file in this subfolder.")
                log_message(log_dir, "alignement_warnings.txt", f"Warning: Common file '{common_file_base_name}' not found in '{sub_dir_full_path}'. Skipping.")


    print("Alignment finish")
    log_message(log_dir, "alignement.txt", "Alignment process finished successfully.")



## Launching Alignment
Please make shure you put your alignment data subfolders in the  /data folder.
each sub folder represent one files corpus to be aligned with the others

In [ ]:
# --- Example Usage (How you would call this function) ---
if __name__ == "__main__":
    # Define your paths here for testing
    base_data_dir = os.path.join(root_path,"data")
    output_aligned_dir = os.path.join(root_path,"aligned_data")
    log_output_dir = os.path.join(root_path,"log")

    # Call the alignment function
    alignement(base_data_dir, output_aligned_dir, log_output_dir)

    print("\nCheck the following directories for results:")
    print(f"  Source: {base_data_dir}")
    print(f"  Aligned Output: {output_aligned_dir}")
    print(f"  Logs: {log_output_dir}")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Log message written to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/not_verified/extracteur_données_audios/alignement/log/alignement.txt
Copied: '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/not_verified/extracteur_données_audios/alignement/data/dataset_incremental_français/phrase_4469.txt' to '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/not_verified/extracteur_données_audios/alignement/aligned_data/dataset_incremental_français/phrase_4469.txt'
Log message written to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/not_verified/extracteur_données_audios/alignement/log/alignement.txt
Copied: '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/not_verified/extracteur_données_audios/alignement/data/dataset_incremental_français/phrase_

In [ ]:
# auto alignment tested

# Visualisation et Evaluation de l'Alignement

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
#Cette fonction permet de visualiser les audios alignés , et de spécifier dans un fichier i l'alignement semble correcte ou pas
import gradio as gr
import os
from typing import List, Tuple

def lire_fichiers_audio(dossier1: str, dossier2: str) -> List[Tuple[str, str, str]]:
    """
    Lit les fichiers audio à partir de deux dossiers et retourne une liste de tuples.
    Les fichiers sont triés par ordre croissant de leur nom.

    Args:
        dossier1: Le chemin du premier dossier.
        dossier2: Le chemin du deuxième dossier.

    Returns:
        Une liste de tuples (nom du fichier, chemin du fichier 1, chemin du fichier 2).
        Seuls les fichiers avec des noms correspondants sont inclus.
        La liste est triée par ordre croissant des noms de fichiers.
    """
    fichiers_dossier1 = set(os.listdir(dossier1))
    fichiers_dossier2 = set(os.listdir(dossier2))
    noms_fichiers_communs = sorted(list(fichiers_dossier1.intersection(fichiers_dossier2)))  # Tri des noms de fichiers
    fichiers_audio = []
    for nom_fichier in noms_fichiers_communs:
        chemin_fichier1 = os.path.join(dossier1, nom_fichier)
        chemin_fichier2 = os.path.join(dossier2, nom_fichier)
        if os.path.isfile(chemin_fichier1) and os.path.isfile(chemin_fichier2):  # Vérifie que ce sont des fichiers
            fichiers_audio.append((nom_fichier, chemin_fichier1, chemin_fichier2))
    return fichiers_audio


def enregistrer_evaluation(chemin_dossier: str, nom_fichier: str, evaluation: int):
    """
    Enregistre l'évaluation dans un fichier texte. Le fichier est créé dans le dossier spécifié.

    Args:
        chemin_dossier: Le chemin du dossier où enregistrer le fichier texte.
        nom_fichier: Le nom du fichier audio évalué.
        evaluation: L'évaluation (-1, 0 ou 1).
    """
    # Créer le dossier s'il n'existe pas
    if not os.path.exists(chemin_dossier):
        os.makedirs(chemin_dossier, exist_ok=True)
    chemin_txt = os.path.join(chemin_dossier, "evaluations.txt")  # Nomme le fichier texte "evaluations.txt"
    # Convertir l'évaluation en texte
    if evaluation == 1:
        evaluation_text = "OK"
    elif evaluation == -1:
        evaluation_text = "Attention"
    else:
        evaluation_text = "Pas encore traité"
    # Écrire le nom du fichier et l'évaluation dans le fichier texte
    with open(chemin_txt, "a") as f:
        f.write(f"{nom_fichier}: {evaluation_text}\n")
    print(f"Évaluation enregistrée pour {nom_fichier} : {evaluation_text}")



def interface_evaluation_audio(dossier1: str, dossier2: str, chemin_dossier: str):
    """
    Crée l'interface Gradio pour évaluer les fichiers audio.
    Les fichiers sont affichés dans l'ordre de leur nom.

    Args:
        dossier1: Le chemin du premier dossier audio.
        dossier2: Le chemin du deuxième dossier audio.
        chemin_dossier: Le chemin du dossier où enregistrer les évaluations.
    """
    fichiers_audio = lire_fichiers_audio(dossier1, dossier2)
    if not fichiers_audio:
        print("Aucun fichier audio correspondant trouvé dans les dossiers.")
        return

    etat = {"index_fichier": 0, "fichiers_audio": fichiers_audio}

    def afficher_audio(etat_dict: dict) -> Tuple[dict, str, str, str]:
        """
        Affiche les fichiers audio à évaluer.

        Args:
            etat_dict: Un dictionnaire contenant l'index du fichier actuel et la liste des fichiers audio.

        Returns:
            Un tuple contenant l'état mis à jour, le nom du fichier et les chemins des fichiers audio.
        """
        if 0 <= etat_dict["index_fichier"] < len(etat_dict["fichiers_audio"]):
            nom_fichier, chemin_fichier1, chemin_fichier2 = etat_dict["fichiers_audio"][etat_dict["index_fichier"]]
            return etat_dict, nom_fichier, chemin_fichier1, chemin_fichier2
        else:
            return etat_dict, "Aucun fichier à afficher", "", ""

    def bouton_suivant(etat_dict: dict, evaluation: int) -> Tuple[dict, gr.Label, gr.Audio, gr.Audio]: # Ajout des types de retour
        """
        Passe à la paire de fichiers audio suivante et enregistre l'évaluation actuelle.
        Les fichiers sont traités dans l'ordre.

        Args:
            etat_dict: Le dictionnaire d'état actuel.
            evaluation: L'évaluation de la paire de fichiers audio actuelle.

        Returns:
            Le nouvel état et les chemins des fichiers audio à afficher.
        """
        if 0 <= etat_dict["index_fichier"] < len(etat_dict["fichiers_audio"]):
            nom_fichier_courant = etat_dict["fichiers_audio"][etat_dict["index_fichier"]][0] # Correction de l'indexation
            enregistrer_evaluation(chemin_dossier, nom_fichier_courant, evaluation)
            etat_dict["index_fichier"] += 1
        etat_val, nom_fichier_audio_str, audio1_path, audio2_path = afficher_audio(etat_dict) # Mise à jour de l'appel
        return etat_val, gr.Label(value=nom_fichier_audio_str), gr.Audio(value=audio1_path), gr.Audio(value=audio2_path) # Retourne les objets Gradio


    with gr.Blocks() as interface:
        etat_gr = gr.State(etat)
        nom_fichier_audio = gr.Label(label="Nom du fichier audio")
        audio1 = gr.Audio(label="Audio Langue 1")
        audio2 = gr.Audio(label="Audio Langue 2")
        with gr.Row():
            bouton_ok = gr.Button("OK")
            bouton_attention = gr.Button("Attention")
        bouton_suivant_fin = gr.Button("Suivant")

        # Initialisation de l'interface
        etat_val, nom_fichier_audio_init, audio1_init, audio2_init = afficher_audio(etat_gr.value)
        nom_fichier_audio.change(fn=lambda initial_value: initial_value, inputs=nom_fichier_audio, outputs=nom_fichier_audio)
        audio1.change(fn=lambda initial_value: initial_value, inputs=audio1, outputs=audio1)
        audio2.change(fn=lambda initial_value: initial_value, inputs=audio2, outputs=audio2)

        # Mise à jour initiale de l'interface avec les premiers fichiers audio
        if etat_gr.value["fichiers_audio"]:
            premier_nom, premier_chemin1, premier_chemin2 = etat_gr.value["fichiers_audio"][0]
            nom_fichier_audio.value = premier_nom  # Mise à jour de la valeur du Label
            audio1.value = premier_chemin1
            audio2.value = premier_chemin2

        bouton_ok.click(
            fn=lambda etat: bouton_suivant(etat, 1), # Removed .value
            inputs=[etat_gr],
            outputs=[etat_gr, nom_fichier_audio, audio1, audio2],
        )
        bouton_attention.click(
            fn=lambda etat: bouton_suivant(etat, -1), # Removed .value
            inputs=[etat_gr],
            outputs=[etat_gr, nom_fichier_audio, audio1, audio2],
        )
        bouton_suivant_fin.click(
            fn=lambda etat: bouton_suivant(etat, 0), # Removed .value
            inputs=[etat_gr],
            outputs=[etat_gr, nom_fichier_audio, audio1, audio2],
        )
    interface.launch(debug=True)



In [ ]:

if __name__ == "__main__":
    # Remplacez ces chemins par les chemins réels de vos dossiers et du fichier CSV
    dossier_langue1 = final_dir_1  # Remplacez par le chemin de votre premier dossier
    dossier_langue2 = final_dir_2  # Remplacez par le chemin de votre deuxième dossier
    chemin_dossier_resultats = log_dir  # Remplacez par le chemin où vous voulez enregistrer le CSV
    interface_evaluation_audio(dossier_langue1, dossier_langue2, chemin_dossier_resultats)


new

In [ ]:
"""

not yet ok , some bugs to fixed .
import gradio as gr
import os
import shutil # Used for creating dummy data for testing
from typing import List, Tuple
from itertools import combinations # Used to generate pairs of subdirectories

# --- Helper Functions ---

def log_message(log_dir: str, filename: str, message: str):
    """
    Writes a message to a log file within the specified log directory.
    Creates the log directory if it doesn't exist.
    """
    os.makedirs(log_dir, exist_ok=True)
    log_file_path = os.path.join(log_dir, filename)
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(message + "\n")
    print(f"Log message written to: {log_file_path}") # For immediate feedback

def lister_dossiers(parent_dossier: str) -> List[str]:
    """Lists full paths of subdirectories directly within a parent folder."""
    if not os.path.isdir(parent_dossier):
        print(f"Warning: Parent directory not found: {parent_dossier}")
        return []
    full_paths = [os.path.join(parent_dossier, d) for d in os.listdir(parent_dossier)]
    return [d for d in full_paths if os.path.isdir(d)]

# --- Modified: Internal helper for getting common files for a specific pair ---
def _get_common_audio_files_for_pair(path_to_dossier1: str, path_to_dossier2: str) -> List[Tuple[str, str, str]]:
    """
    Internal helper to find common audio files (by name) between two specific directories.
    Returns a list of (file_name, full_path_in_dossier1, full_path_in_dossier2) tuples.
    """
    if not os.path.isdir(path_to_dossier1) or not os.path.isdir(path_to_dossier2):
        print(f"Warning: One or both directories not found: '{path_to_dossier1}' or '{path_to_dossier2}'. Skipping this pair.")
        return []

    # Get base names of files in each directory, filtering for actual files
    files_dossier1 = set(f for f in os.listdir(path_to_dossier1) if os.path.isfile(os.path.join(path_to_dossier1, f)))
    files_dossier2 = set(f for f in os.listdir(path_to_dossier2) if os.path.isfile(os.path.join(path_to_dossier2, f)))

    # Find common file names and sort them
    common_file_names = sorted(list(files_dossier1.intersection(files_dossier2)))

    audio_pairs = []
    for nom_fichier in common_file_names:
        chemin_fichier1 = os.path.join(path_to_dossier1, nom_fichier)
        chemin_fichier2 = os.path.join(path_to_dossier2, nom_fichier)
        audio_pairs.append((nom_fichier, chemin_fichier1, chemin_fichier2))
    return audio_pairs

# --- New Function: Collects all unique pairs of subdirectories and their common files ---
def collect_all_evaluation_items(aligned_data_root_dir: str) -> List[Tuple[str, str, str, str, str]]:
    """
    Discovers all unique pairs of subdirectories within aligned_data_root_dir
    and finds common audio files for each pair.

    Returns a flat list of tuples, each representing one item to evaluate:
    (sub_dir_name_1, sub_dir_name_2, file_name, path_to_file_in_dir1, path_to_file_in_dir2)
    """
    all_sub_dir_full_paths = lister_dossiers(aligned_data_root_dir)
    # Extract just the directory names (e.g., 'speaker_A') from full paths
    all_sub_dir_names = [os.path.basename(p) for p in all_sub_dir_full_paths]

    if len(all_sub_dir_names) < 2:
        print(f"Error: Less than 2 subdirectories found in '{aligned_data_root_dir}'. Cannot form pairs for evaluation.")
        return []

    print(f"Found {len(all_sub_dir_names)} subdirectories: {all_sub_dir_names}")
    evaluation_items = []

    # Generate all unique combinations of 2 subdirectories
    # e.g., for [A, B, C] -> (A, B), (A, C), (B, C)
    for dir1_name, dir2_name in combinations(all_sub_dir_names, 2):
        print(f"Processing pair: ({dir1_name}, {dir2_name})")
        path_to_dossier1 = os.path.join(aligned_data_root_dir, dir1_name)
        path_to_dossier2 = os.path.join(aligned_data_root_dir, dir2_name)

        # Get common files for this specific pair of directories
        common_files_for_pair = _get_common_audio_files_for_pair(path_to_dossier1, path_to_dossier2)

        # Add each common file found in this pair to our main evaluation list
        for file_name, path_file1, path_file2 in common_files_for_pair:
            evaluation_items.append((dir1_name, dir2_name, file_name, path_file1, path_file2))

    # Sort the entire list of evaluation items for consistent order
    evaluation_items.sort(key=lambda x: (x[0], x[1], x[2]))
    print(f"Total {len(evaluation_items)} audio pairs identified for evaluation.")
    return evaluation_items

# --- Modified: `enregistrer_evaluation` to include directory names ---
def enregistrer_evaluation(chemin_dossier: str,
                           dir1_name: str,
                           dir2_name: str,
                           nom_fichier: str,
                           evaluation: int):
    """
    Records the evaluation in a text file, specifying the pair of folders
    being compared for a given audio file.
    """
    os.makedirs(chemin_dossier, exist_ok=True)
    chemin_txt = os.path.join(chemin_dossier, "evaluations.txt")

    # Convert evaluation to text
    if evaluation == 1:
        evaluation_text = "OK"
    elif evaluation == -1:
        evaluation_text = "Attention"
    else: # evaluation == 0 or other
        evaluation_text = "Pas encore traité"

    # Write the file name, the specific pair, and the evaluation to the text file
    with open(chemin_txt, "a", encoding="utf-8") as f:
        f.write(f"Pair ({dir1_name} vs {dir2_name}) - Fichier '{nom_fichier}': {evaluation_text}\n")
    print(f"Évaluation enregistrée pour Pair ({dir1_name} vs {dir2_name}) - Fichier '{nom_fichier}' : {evaluation_text}")


# --- Main Gradio Interface Function ---
def interface_evaluation_audio(aligned_data_root_dir: str, evaluation_output_dir: str):
    """
    Creates a Gradio interface to automatically iterate through and evaluate
    common audio files for all unique pairs of subdirectories within a root directory.
    """
    # Collect all items to be evaluated (pairs of subdirectories and common files)
    evaluation_items = collect_all_evaluation_items(aligned_data_root_dir)

    if not evaluation_items:
        print(f"No audio file pairs found for evaluation in '{aligned_data_root_dir}'.")
        with gr.Blocks() as interface:
            gr.Markdown("## Aucun fichier audio aligné trouvé pour l'évaluation.")
            gr.Markdown(f"Vérifiez que le dossier `{aligned_data_root_dir}` contient au moins deux sous-dossiers et des fichiers audio communs entre eux.")
        interface.launch(debug=True)
        return

    # Initial state for the Gradio app
    # `index_courant` tracks the current item in `evaluation_items`
    etat = {"index_courant": 0, "evaluation_items": evaluation_items}

    def _display_current_audio_pair(etat_dict: dict) -> Tuple[dict, str, gr.Audio, gr.Audio, str, str]:
        """
        Helper function to get data for the current audio pair and format it for Gradio outputs.
        Returns: updated state dict, file display string, audio1 component, audio2 component,
                 label for audio1, label for audio2.
        """
        if 0 <= etat_dict["index_courant"] < len(etat_dict["evaluation_items"]):
            dir1_name, dir2_name, file_name, path_file1, path_file2 = etat_dict["evaluation_items"][etat_dict["index_courant"]]
            status_message = f"Évaluation {etat_dict['index_courant'] + 1} sur {len(etat_dict['evaluation_items'])}"
            full_file_display_name = f"Fichier: '{file_name}' (de '{dir1_name}' vs '{dir2_name}')"

            # Return Gradio Audio components with updated values and desired labels
            return (etat_dict, full_file_display_name,
                    gr.Audio(value=path_file1, label=f"Audio ({dir1_name})", type="filepath"),
                    gr.Audio(value=path_file2, label=f"Audio ({dir2_name})", type="filepath"),
                    f"Audio ({dir1_name})", # Actual label string for audio1
                    f"Audio ({dir2_name})"  # Actual label string for audio2
                   )
        else:
            # End of evaluation
            status_message = "Évaluation terminée. Tous les fichiers ont été traités."
            return (etat_dict, "FIN",
                    gr.Audio(value=None, label="Audio 1"), # Clear audio player
                    gr.Audio(value=None, label="Audio 2"), # Clear audio player
                    "Audio 1", # Default label
                    "Audio 2"  # Default label
                   )

    def _on_button_click(etat_dict: dict, evaluation_value: int) -> Tuple[dict, gr.Label, gr.Audio, gr.Audio, gr.Markdown, gr.Audio, gr.Audio]:
        """
        Handles button clicks: records evaluation, moves to the next item, and updates UI.
        """
        # Record evaluation for the *current* file before moving to the next
        if 0 <= etat_dict["index_courant"] < len(etat_dict["evaluation_items"]):
            dir1_name, dir2_name, nom_fichier_courant, _, _ = etat_dict["evaluation_items"][etat_dict["index_courant"]]
            enregistrer_evaluation(evaluation_output_dir, dir1_name, dir2_name, nom_fichier_courant, evaluation_value)

        # Move to the next item
        etat_dict["index_courant"] += 1

        # Get data for the next display
        updated_state, file_display_name, audio1_comp, audio2_comp, label1_str, label2_str = _display_current_audio_pair(etat_dict)

        # Return updated Gradio components
        return (updated_state,
                gr.Label(value=file_display_name), # Update the file name label
                audio1_comp, # Update audio1 player
                audio2_comp, # Update audio2 player
                gr.Markdown(value=label1_str.replace('Audio (', 'Fichier 1: Audio (')), # Update status/markdown
                audio1_comp, # This is here to allow updating the label property if needed
                audio2_comp) # This is here to allow updating the label property if needed


    # --- Gradio Interface Definition ---
    with gr.Blocks(title="Évaluation d'Alignement Audio") as interface:
        # Gradio State component to hold our internal state
        etat_gr = gr.State(etat)

        gr.Markdown("## Outil d'Évaluation Automatique d'Alignement Audio")
        gr.Markdown(f"Le dossier racine des données alignées est : **`{aligned_data_root_dir}`**")
        gr.Markdown(f"Les évaluations seront enregistrées dans : **`{evaluation_output_dir}/evaluations.txt`**")
        gr.Markdown("Vous allez évaluer séquentiellement toutes les paires uniques de sous-dossiers et leurs fichiers audio communs.")

        # Display for current file name and pair
        nom_fichier_audio_display = gr.Label(label="Fichier en cours d'évaluation")

        # Display for current evaluation progress (e.g., "1/100")
        status_display = gr.Markdown(value=f"Évaluation 1 sur {len(evaluation_items)}")

        # Audio players - defined once, values and labels updated dynamically
        with gr.Row():
            # Using render=False so we can manually place them with .render()
            audio1 = gr.Audio(label="Audio 1 (Dossier 1)", type="filepath", render=False)
            audio2 = gr.Audio(label="Audio 2 (Dossier 2)", type="filepath", render=False)

            with gr.Column():
                gr.Markdown("### Audio 1:")
                audio1.render() # Place the first audio player
            with gr.Column():
                gr.Markdown("### Audio 2:")
                audio2.render() # Place the second audio player

        # Evaluation buttons
        with gr.Row():
            bouton_ok = gr.Button("OK (Alignement correct)", variant="primary")
            bouton_attention = gr.Button("Attention (Problème d'alignement)", variant="secondary")
            bouton_suivant_fin = gr.Button("Passer (Non évalué)", variant="tertiary")

        # --- Initial Setup for the First Audio Pair ---
        # Call _display_current_audio_pair once to set the initial UI state
        initial_state_val, initial_file_name_str, initial_audio1_comp, initial_audio2_comp, \
            initial_label1_str, initial_label2_str = _display_current_audio_pair(etat_gr.value)

        # Update Gradio component values with the initial data
        nom_fichier_audio_display.value = initial_file_name_str
        audio1.value = initial_audio1_comp.value
        audio1.label = initial_label1_str # Set initial label for player 1
        audio2.value = initial_audio2_comp.value
        audio2.label = initial_label2_str # Set initial label for player 2


        # --- Button Click Event Handlers ---
        # When a button is clicked, call _on_button_click with the current state and evaluation value.
        # Outputs must match the return types of _on_button_click.
        # We need to return audio1_comp and audio2_comp *again* in outputs for their labels to update
        # because the label property is part of the component itself.
        bouton_ok.click(
            fn=lambda current_state: _on_button_click(current_state, 1),
            inputs=[etat_gr],
            outputs=[etat_gr, nom_fichier_audio_display, audio1, audio2, status_display, audio1, audio2]
        )
        bouton_attention.click(
            fn=lambda current_state: _on_button_click(current_state, -1),
            inputs=[etat_gr],
            outputs=[etat_gr, nom_fichier_audio_display, audio1, audio2, status_display, audio1, audio2]
        )
        bouton_suivant_fin.click(
            fn=lambda current_state: _on_button_click(current_state, 0),
            inputs=[etat_gr],
            outputs=[etat_gr, nom_fichier_audio_display, audio1, audio2, status_display, audio1, audio2]
        )

    interface.launch(debug=True, share=False) # share=True if you need a public link


In [ ]:
""" Not yet ok"
if __name__ == "__main__":
    aligned_data_root = os.path.join(root_path,'aligned_data')
    evaluation_output_folder = os.path.join(root_path,'log')

    # Launch the Gradio interface
    interface_evaluation_audio(aligned_data_root, evaluation_output_folder)

Found 2 subdirectories: ['anglais', 'shupamom']
Processing pair: (anglais, shupamom)
Total 404 audio pairs identified for evaluation.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


In [ ]:
"""import os

def remplacer_t_par_a(repertoire):
    """
    Remplace toutes les occurrences de 't' par 'a' dans les noms des fichiers .mp3
    dans le répertoire spécifié.

    Args:
        repertoire (str): Le chemin d'accès au répertoire contenant les fichiers.
    """
    for fichier in os.listdir(repertoire):
        if fichier.endswith(".wav"):
            nouveau_nom = fichier.replace('t', 'a')  # Remplace 't' par 'a'
            ancien_chemin = os.path.join(repertoire, fichier)
            nouveau_chemin = os.path.join(repertoire, nouveau_nom)
            os.rename(ancien_chemin, nouveau_chemin)
            print(f'Renommé: "{ancien_chemin}" en "{nouveau_chemin}"')

# Exemple d'utilisation
remplacer_t_par_a("/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Renommé: "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio/t_s_88_v_10.wav" en "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio/a_s_88_v_10.wav"
Renommé: "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio/t_s_13_v_37.wav" en "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio/a_s_13_v_37.wav"
Renommé: "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio/t_s_43_v_11.wav" en "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_texte_audio_anglais/audio/a_s_43_v_11.wav"
Renommé: "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/dataset_versets_coran_tex

# End Thanks you